In [ ]:
import random
from typing import TypedDict, List, Annotated
import operator
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from IPython.display import Image, display
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("OPENAI_ENDPOINT")
model_name = "gpt-4o-mini"
temp=0.0

llm = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model=model_name,
    temperature=temp
)

## **Config Object Across Invoke Calls**
- The config object in LangGraph is a RunnableConfig instance (essentially a dictionary) that carries immutable, runtime-specific information across the graph's execution (across all invoke calls)
- It is primarily used for system-level settings, tracing, and passing configuration values that should not be part of the mutable agent state.
- Unlike the State object lifecycle that is limited to each invoke call, the Config object lives across multiple invoke calls
- It is used to manage threads, metadata, config variables, etc.

In [13]:
class State(MessagesState):
    question:str

def my_node(state: State, config: RunnableConfig):
    question = state["question"]
    messages = state["messages"]
    name = None
    if config:
        name = config.get("configurable").get("name")

    if name and not messages:
        messages.append(
            SystemMessage(
                "Help the following user. \n"
                f"Name={name} \n"
                "Give an initial greeting before responding."
            )
        )

    messages.append(HumanMessage(question))
    response = llm.invoke(messages)
    messages.append(response)

    return {"messages": messages}

workflow = StateGraph(State)
workflow.add_node("model", my_node)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

In [14]:
run_config = {
    "thread_id": "user_session_123",
    "tags": ["experiment-A", "deployment-42"],
    "configurable": {
        "name": "James"
    }
}

In [15]:
graph = workflow.compile()
result = graph.invoke(input={"question": "What's the name of Ash's first pokémon?"}, config=run_config)

In [16]:
for message in result['messages']:
    message.pretty_print()

================================ System Message ================================

Help the following user. 
Name=James 
Give an initial greeting before responding.
================================ Human Message =================================

What's the name of Ash's first pokémon?
================================== Ai Message ==================================

Hello, James! Ash's first Pokémon is Pikachu. He received Pikachu from Professor Oak at the beginning of his journey in the Pokémon anime. If you have any more questions about Pokémon or anything else, feel free to ask!
